In [ ]:
import os

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Dense, RNN,LSTM
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.layers import Input,Flatten,Reshape
from tensorflow.keras.layers import Activation, Dropout,BatchNormalization
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.optimizers import Adam, RMSprop

from PIL import Image
from PIL import ImageDraw

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential , Model,layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten,UpSampling2D,Activation



import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
import matplotlib.cm as cm

from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE


import pickle

# 이미지 전처리

In [ ]:
dataset =[] 
for file in file_list:
    img_path = './users/seounghyun.lee/mood_up/images_0411/' + file
    img = Image.open(img_path)
    img = img.convert('RGB')
    img = img.resize((132,68))
    

    data = np.asarray(img)
    data = data[2:68-2,2:132-2]
    dataset.append(data)
dataset = np.array(dataset)

In [ ]:
x_train,x_test = train_test_split(dataset,test_size = 0.2,random_state = 42)

In [ ]:
def dataset_asnp(dataset,dataset_shape):
    x_np = np.zeros(list(dataset_shape))
    y_np = np.zeros(dataset_shape[0])
    for i, z in enumerate(dataset):
        x_np[i] = z
    return x_np

x_train = dataset_asnp(x_train, shape_train)
x_test= dataset_asnp(x_test, shape_test)


In [ ]:
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

In [ ]:
def plot_show(data, p_row, p_col, size, title = False):
    fig,axis = plt.subplots(p_row, p_col, figsize = (p_col*size, p_row*size),
                          subplot_kw = {'xticks':[],'yticks':[]},
                          gridspec_kw = dict(hspace=0.1,wspace=0.1))
    for i ,ㅡ ax in enumerate(axis.flat):
        ax.imshow(data[i],cmap = 'gray')
    if title:
        fig.suptitle(title)

In [ ]:
plot_show(x_train[:10].reshape(10,64,128, 3),2,5,1.5,'train')
plt.savefig('trin_dataset.png')
plot_show(x_test[:10].reshape(10,64,128, 3),2,5,1.5,'test')
plt.savefig('test_dataset.png')

# 차원축소 인코딩

In [ ]:
input_layer = Input(shape=(64, 128, 3,))
hidden_encoding = Conv2D(
    32, # Number of layers or filters in the weight matrix
    (3, 3), # Shape of the weight matrix
    padding='same', # How to apply the weights to the images
)(input_layer)
hidden_encoding = BatchNormalization()(hidden_encoding)
hidden_encoding = Activation('relu')(hidden_encoding)
encoded = MaxPooling2D((2, 2))(hidden_encoding)
hidden_decoding = Conv2D(
    32, # Number of layers or filters in the weight matrix
    (3, 3), # Shape of the weight matrix
  
    padding='same', # How to apply the weights to the images
)(encoded)
hidden_decoding = BatchNormalization()(hidden_decoding)
hidden_decoding = Activation('relu')(hidden_decoding)
upsample_decoding = UpSampling2D((2, 2))(hidden_decoding)
decoded = Conv2D(
    3, # Number of layers or filters in the weight matrix
    (3, 3), # Shape of the weight matrix
    activation='sigmoid',
    padding='same', # How to apply the weights to the images
)(upsample_decoding)
autoencoder = Model(input_layer, decoded)
autoencoder.summary()

In [ ]:
cp = tf.keras.callbacks.ModelCheckpoint(filepath="autoencoder_fraud.h5",
                               mode='min', monitor='val_loss', verbose=2, save_best_only=True)

In [ ]:
epochs = 100
batch_size = 128

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = autoencoder.fit(x_train, x_train, 
                          epochs=epochs, 
                          batch_size=batch_size, 
                          shuffle=True, 
                          validation_data=(x_test, x_test),
                          callbacks =[cp_callback]
                         ).history

In [ ]:
plt.plot(history['loss'], linewidth=2, label='Train')
plt.plot(history['val_loss'], linewidth=2, label='Test')
plt.legend(loc='upper right')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.savefig('loss.png')
plt.show()

In [ ]:
encoder = Model(input_layer, encoded)
total_data = np.vstack((x_train,x_test))
encoded_imgs = encoder.predict(total_data)

# 군집화

In [ ]:
encoded_imgs = encoder.predict(total_data)

In [ ]:

#엘보우 기법
from sklearn.cluster import KMeans


ks = range(1,11)
sse = []

for k in ks:
    model = KMeans(n_clusters=k, init = 'k-means++',random_state = 0)
    model.fit(encoded_imgs)
    sse.append(model.inertia_)


plt.figure(figsize=(4, 4))

plt.plot(ks, sse, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('sse')
plt.xticks(ks)
plt.savefig('elbow')
plt.show()

In [ ]:
#실루엣 지수 확인(1) : 각 클러스터별 실루엣 지수 나오게 하는 
def visualize_silhouette(cluster_lists, X_features): 
    
    from sklearn.datasets import make_blobs
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math
    
   
    n_cols = len(cluster_lists)
    
   
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)
    
   
    for ind, n_cluster in enumerate(cluster_lists):
        
       
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(X_features)
        
        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)
        
        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
        
       
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()
            
            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i
            
            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10
            
        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")
        plt.savefig(f'{i}silhouette')

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=500, n_features=2, centers=4, cluster_std=1, \
                  center_box=(-10.0, 10.0), shuffle=True, random_state=1)  
visualize_silhouette([ 4,5,6,7], X)

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(
    n_samples=971,
    n_features=3,
    centers=5,
    cluster_std=0.7,
    center_box=(-10.0, 10.0),
    shuffle=True,
    random_state=1)

visualize_silhouette([ 4,5,6,7], X)

In [ ]:
#실루엣 지수 확인(2) : 각 클러스터별 군집 정도 scatter로 확인
X = encoded_imgs

# Generating the sample data from make_blobs
# This particular setting has one distinct cluster and 3 clusters placed close
# together.
X, y = make_blobs(
    n_samples=971,
    n_features=3,
    centers=5,
    cluster_std=0.7,
    center_box=(-10.0, 10.0),
    shuffle=True,
    random_state=1,
)  # For reproducibility

range_n_clusters = [i for i in range(2,11)]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
#     clusterer = KMeans(n_clusters=n_clusters, n_init="auto", random_state=10)
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(
        X[:, 0], X[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
    )

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )
    plt.savefig(f'my_plot{i}.png')

plt.show()

In [ ]:
#군집화
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, init = 'k-means++',random_state=42)
kmeans.fit(encoded_imgs)

In [ ]:
data_cluster = kmeans.fit(encoded_imgs)

# 시각화

In [ ]:
for i in range(n_clusters):

    cluster_imgs = total_data[kmeans.labels_ == i][i]
    n_images = len(cluster_imgs)
    n_rows = ceil(n_images / 5)
    fig, axes = plt.subplots(n_rows, 6, figsize=(15, n_rows*3))
    for j, img in enumerate(cluster_imgs):
        img = np.array(img*255, dtype=np.uint8)  # 이미지 데이터 타입 변환
        img = Image.fromarray(img)  # PIL 이미지 객체로 변환
        img = img.resize((128, 64))  # 이미지 크기 조절
        ax = axes.flat[j]
        ax.imshow(img)
        ax.axis('off')
    for j in range(n_images, n_rows*6):
        ax = axes.flat[j]
        ax.set_visible(False)
    plt.savefig(f'clustering_img{i+1}.png')
    plt.show()

In [ ]:
tsne = TSNE(n_components = 2, random_state = 42)
x_tsne = tsne.fit_transform(data_test)

plt.figure(figsize=(20,8))
for c in np.unique(data_cluster.labels_):
    plt.scatter(tsne_clust[data_cluster.labels_ == c,0], tsne_clust[data_cluster.labels_ == c,1], label='Cluster {}'.format(c+1),s = 150)

# plt.axis('off')
plt.savefig('clust_check')
plt.legend()

In [ ]:
cluster_images = [[] for _ in range(n_clusters)]
for label, img in zip(kmeans.labels_, total_data):
    cluster_images[label].append(img)

# 차원축소 오토 인코딩

- 군집별그룹화

In [ ]:
cluster_images_4parts = [[[]for _ in range(4)] for _ in range(n_clusters)]
temp = []
for idx,images in enumerate(cluster_images):
    img_parts = []
    for img in images:
        parts = np.split(img, 16, axis=0) # 이미지를 세로로 4개로 분할

        now = 0
        for i, part in enumerate(parts):
            if i in [1,5,9,13]:
                cluster_images_4parts[idx][now].append(part)
                now += 1

- 군집별 모든 색상 추출

In [ ]:
 

# 군집 내 각 행의 이미지에서 추출된 색상을 저장할 리스트

color_list = []


for i in range(n_clusters):
    cluster = cluster_images_4parts[i] # 현재 군집
    print('cluster',len(cluster))
    
    row_color_list = [[]for _ in range(4)]  # 현재 군집 내 각 행에서 추출된 색상을 저장할 리스트
    now = 0
    
    for j in range(4):
        row = cluster[j]
#         print('now',j)
#         print('--------------------------------------')
#         print(row)
        for img in row:
            row_color = [] # 현재 행에서 추출된 색상을 저장할 리스트
#             plt.imshow(img)
            plt.show()
            img = (img* 255).astype(np.uint8)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # 이미지를 RGB 색상 공간으로 변환
#             display_image(img)
            colors = np.unique(img.reshape(-1, img.shape[2]), axis=0) # 이미지에서 모든 색상 추출
  

            for color in colors:
                if not np.all(color == [0,0,0]):
                    color_img = np.zeros((50, 50, 3), dtype=np.uint8)
                    color_img[:, :, :] = color
#                         print(f"\tRow {j}:")
#                     display_image(color_img)
                    row_color.append(color.tolist()) # 추출된 색상을 리스트에 추가
#                     print(row_color)
#                     print('rowcolor',len(row_color))
            row_color_list[j].extend(row_color)
#             print('row_color_list',row_color_list)
    color_list.append(row_color_list)

In [ ]:
def display_image(img):
    fig, ax = plt.subplots(figsize=(1, 1))
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax.axis('off')
    plt.show()

- 군집별 색상 추천(빈도수 확인)

In [ ]:
from collections import Counter
recommand_list= []

for i in range(n_clusters):
    print(f"Cluster {i+1}:")
    cluster = color_list[i]
    top_list = [[]for _ in range(4)]
#     print(cluster)
    for idx, row in enumerate(cluster):
# #       
        color_counts = np.unique(row, axis=0, return_counts=True)
        top_indices = np.argsort(color_counts[1])[::-1][:6]
        top_colors = color_counts[0][top_indices]
        for color in top_colors:
            color_img = np.zeros((50, 50, 3), dtype=np.uint8)
            color_img[:, :, :] = color
#             print('check',color_img.shape)
#             print(f"\tRow {idx}:")
            display_image(color_img)
            top_list[idx].append(color.tolist())
        print('toplist',top_list)
    recommand_list.append(top_list)
#     print(recommand_list)
print(len(recommand_list))

- 시각화

In [ ]:
# def display_color_grid(x):
for idx, cluster in enumerate(recommand_list):
    fig, axes = plt.subplots(nrows=len(cluster), ncols=6, figsize=(10,10))
    
    for i in range(len(cluster)):
        for j in range(6):
            color_img = np.zeros((50, 50, 3), dtype=np.uint8)
            color = cluster[i][j]
            color_img[:, :, :] = color
#             print('check', color)
            axes[i, j].imshow(cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB))
            axes[i, j].axis('off')
            
    print(f"Cluster {idx+1}:")
    plt.tight_layout()
    plt.savefig(f'cluster{idx+1}.png')
    plt.show()